<a href="https://colab.research.google.com/github/KrisSandy/TheSilenceOfNLP/blob/master/Spell_Checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spell Checker

This assignment will involve the creation of a spellchecking system and an evaluation of its performance. You may use the code snippets provided in Python for completing this or you may use the programming language or environment of your choice

Please start by downloading the corpus `holbrook.txt` from Blackboard

The file consists of lines of text, with one sentence per line. Errors in the line are marked with a `|` as follows

    My siter|sister go|goes to Tonbury .
    
In this case the word 'siter' was corrected to 'sister' and the word 'go' was corrected to 'goes'.

In some places in the corpus two words maybe corrected to a single word or one word to a multiple words. This is denoted in the data using underscores e.g.,

    My Mum goes out some_times|sometimes .
    
For the purpose of this assignment you do not need to separate these words, but instead you may treat them like a single token.

*Note: you may use any functions from NLTK to complete the assignment. It should not be necessary to use other libraries and so please consult with us if your solution involves any other external library. If you use any function from NLTK in Task 6 please include a brief description of this function and how it contributes to your solution.*

## Text Parser

Write a parser that can read all the lines of the file `holbrook.txt` and print out for each line the original (misspelled) text, the corrected text and the indexes of any changes. The indexes refers to the index of the words in the sentence. In the example given, there is only an error in the 10th word and so the list of indexes is [9]. It is not necessary to analyze where the error occurs inside the word.

Then split your data into a test set of 100 lines and a training set.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
lines = open(r'/content/gdrive/My Drive/GYE06/CT5120_NLP/Data/holbrook.txt').readlines()
data = []
# Write your code here

for line in lines:
    words = line.split()
    data_sent_dict = {"original": [], "corrected": [], "indexes": []}
    for i, word in enumerate(words):
        if word.find("|") != -1:
            word_epair = word.split("|")
            data_sent_dict["original"].append(word_epair[0])
            data_sent_dict["corrected"].append(word_epair[1])
            data_sent_dict["indexes"].append(i)
        else:
            data_sent_dict["original"].append(word)
            data_sent_dict["corrected"].append(word)
    data.append(data_sent_dict)

assert (data[2] == {
    'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'],
    'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'],
    'indexes': [9]
})

The counts and assertions given in the following sections are based on splitting the training and test set as follows

In [0]:
  test = data[:100]
  train = data[100:]

## Calculate Frequency: 
Calculate the frequency (number of occurrences), *ignoring case*, of all words and their unigram probability from the corrected *training* sentences.

*Hint: use `Counter` to implement this so it may be called many times*

In [0]:
from collections import Counter

word_dictionary = [w.lower() for s in train for w in s["corrected"]]
word_counts = Counter(word_dictionary)
tot_words = sum([len(s["corrected"]) for s in train])


def unigram(word):
    # Write your code here.
    return word_counts[word.lower()]


def prob(word):
    return float(unigram(word)) / tot_words
  
# Test your code with the following
assert(unigram("me")==87)

## Calculate Distance: 
[Edit distance](https://en.wikipedia.org/wiki/Edit_distance) is a method that calculates how similar two strings are to one another by counting the minimum number of operations required to transform one string into the other. There is a built-in implementation in NLTK that works as follows:


In [0]:
from nltk.metrics.distance import edit_distance

# Edit distance returns the number of changes to transform one word to another
print(edit_distance("hello", "hi"))

4


Write a function that calculates all words with *minimal* edit distance to the misspelled word. You should do this as follows

1. Collect the set of all unique tokens in `train`
2. Find the minimal edit distance, that is the lowest value for the function `edit_distance` between `token` and a word in `train`
3. Output all unique words in `train` that have this same (minimal) `edit_distance` value

*Do not implement edit distance, use the built-in NLTK function `edit_distance`*

In [0]:
def get_candidates(token):
    # Write your code here.

    token_distances = {k: edit_distance(token.lower(), k) for k in word_dictionary}
    candidates = [k for k, v in token_distances.items() if v == min(token_distances.values())]

    return candidates
        
# Test your code as follows
assert get_candidates("minde") == ['mind', 'mine']

## Correct Sentence:

Write a function that takes a (misspelled) sentence and returns the corrected version of that sentence. The system should scan the sentence for words that are not in the dictionary (set of unique words in the training set) and for each word that is not in the dictionary choose a word in the dictionary that has minimal edit distance and has the highest *unigram probability*. 

*Your solution to this should involve `get_candidates`*


In [0]:
def correct(sentence):
    # Write your code here

    corrected_sentence = []

    for word in sentence:
        if word.lower() not in word_dictionary:
            candidates = get_candidates(word)
            if len(candidates) == 1:
                corrected_sentence.append(candidates[0])
            else:
                candidates_prob = {c: prob(c) for c in candidates}
                corrected_sentence.append(max(candidates_prob, key=candidates_prob.get))
        else:
            corrected_sentence.append(word)

    return corrected_sentence

assert(correct(["this","whitr","cat"]) == ['this','white','cat'])   

## Accuracy: 
Using the test corpus evaluate the *accuracy* of your method, i.e., how many words from your system's output match the corrected sentence (you should count words that are already spelled correctly and not changed by the system).

In [0]:
def accuracy(test):
    # Write your code here

    n = 0.0
    n_correct = 0.0
    for sentence in test:
        n += len(sentence['corrected'])
        sentence_corrected = correct(sentence['original'])
        for (word_a, word_c) in zip(sentence['corrected'], sentence_corrected):
            if word_a == word_c:
                n_correct += 1

    print("{} words matched out of {} words in corrected sentences".format(n_correct, n))

    return n_correct/n

print("Accuracy of the current Spell Checker : {}".format(accuracy(test)))

946.0 words matched out of 1129.0 words in corrected sentences
Accuracy of the current Spell Checker : 0.837909654562


The above spell checker algorithm has an accuracy of 83.7% which uses minimum edit distances and unigram probabilities for correcting the words.

## Modifications

Consider a modification to your algorithm that would improve the accuracy of the algorithm developed in Task 3 and 4

* You may resources beyond those provided here.
* You must **not use the test data** in this task.
* Provide a short text describing what you intend to do and why. 
* Full marks for this section may be obtained without an implementation, but an implementation is preferred.
* Your implementation should not consist of more than 50 lines of code

Please note this task is marked according to: demonstration of knowledge from the lecutures (10), originality and appropriateness of solution (10), completeness of description (10) and technical correctness (5)


### Improvements to current Spell Checker

1.   Ignore numbers from the correction algorithm
2. inflection 
3. lemma
4. Named entity reognision (ignore NNPs)
5. Tenses



In [0]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection u'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzip

True

#### 1. Named Entity recognision
One of the problems with the above spell checker is that it is trying to correct proper nouns and named entities. For example word 'NIGEL' which is a proper noun is changed to 'night'.  This senario can be mitigated by identifying named entities/proper nouns. 

nltk's pos_tag function will identify proper nouns by tagging them with NNP. Hence tagging the entire sentence and ignoring any proper nouns (NNP tags) will avoid unnecessary correction.

We can also use nltk's ne_chunk method to find named entity. ne_chunk gives a Tree structure with a lable defining the entity type. Function is_named_entity in the below code does the named entity check using ne_chunk method and returns True if a word is named Entity.

In [0]:
from nltk import ne_chunk

def is_named_entity(word):
    wordl = []
    wordl.append(word)
    entity = nltk.ne_chunk(nltk.pos_tag(wordl))
    for chunk in entity:
        if hasattr(chunk, 'label'):
            return True
    return False
  
print(is_named_entity('NIGEL'))

True


#### 2. Numbers identification

Numeric entity can be ignored from the spell check. In order to do this, we can use nltk's pos_tags method which tags numbers with 'CD' tag. However pos_tag is not working for negative numbers, in which case they are identified as NN. Hence in below function, I have used both pos_tags method and also regular expressions to identify numeric entities. pos_tags also detects dates etc which can be ignored as well. 

In [0]:
import re
numeric_re = re.compile(r"[+-]?\d+")

def is_numeric(word):
    wordl = []
    wordl.append(word)
    tags = nltk.pos_tag(wordl)
    
    if re.match(numeric_re, word) is not None or tags[0][1] == 'CD':
        return True
    
    return False

print(is_numeric("-48"))

True


#### 3. Use Wordnet to check correct words

One of the problems with the spell checker is that it is correcting the correct words if it is not present in the training dictionary. Hence in order to avoid this, we can use nlkt wordnet corpus and compare each word with this corpus before passing it to the spell checker. The implementation is done in correct2 function below

#### 4. Lemmatizing

Lemmatizing helps in bringing the same words with different tense in one form. For example saturdays is not identified by the spell checker because this form of the word is not present in the dictionary trained by training data. This may be the case with words present in the dictionary as well. Hence nltk's lemmarizer is used to convert these words into their actual form for easy matching with dictionary.

In [0]:
from nltk.stem import WordNetLemmatizer

word_dictionary_lemma = []
lemmatizer = WordNetLemmatizer()

for word in word_dictionary:
  word_dictionary_lemma.append(lemmatizer.lemmatize(word))
  word_dictionary_lemma.append(word)

#### 5. Linear Interpolation (Bigrams Model)

After getting the candidate words by calculating minium distance of words, we are taking a simple unigram probability to choose from multiple words. This may result in using the most frequent word which is out of context in the sentence as replacement. To overcome this, we can use bigrams which calculates the probabilities by dividing matched bigrams pair by total number of bigrams. However as our training data is small, we might not get bigrams for most of the pairs which results in zero probability. In such cases we can use unigram probabilities. To combine both approaches, I have used Linear Interpolation method in which both bigram model and trigram model is used and weights are given to each model. These weights ($lambda1, lambda2$) can be adjusted to tune our model

$P(word) = lambda1*P(unigram) + lambda2*P(bigram)$ where $ lambda1 + lambda2 = 1$

This model can be extended to trigrams.

In [0]:
from nltk.probability import ConditionalFreqDist

all_words = [w.lower() for s in train for w in s["corrected"]]
bigrams_freqDist = ConditionalFreqDist(nltk.bigrams(all_words))


def bigrams(word1, word2):
    val = bigrams_freqDist[word1.lower()].get(word2.lower())
    if val is not None:
      return val
    else:
      return 0
    

def prob_bigrams(word1, word2):
#   +1 is added to the denominator for smoothing
    return float(bigrams(word1, word2)) / (sum(bigrams_freqDist[word1.lower()].values()) + 1)


In [0]:
from nltk.corpus import wordnet
  
def correct2(sentence):

    lambda1 = 0.5
    lambda2 = 0.5
    sentence_tags = nltk.pos_tag(sentence)
    corrected_sentence = []
    prev_word = ''
    
    for (word, tag) in sentence_tags:
        if is_numeric(word) or is_named_entity(word) or tag == "NNP":
            corrected_sentence.append(word)
        elif word.lower() in wordnet.words() or \
             lemmatizer.lemmatize(word.lower()) in wordnet.words():
            corrected_sentence.append(word)
        elif word.lower() not in word_dictionary_lemma and \
             lemmatizer.lemmatize(word.lower()) not in word_dictionary_lemma:
            candidates = get_candidates(word)
            if (len(candidates) == 1):
                corrected_sentence.append(candidates[0])
            else:
                candidates_uprob = {c: prob(c) for c in candidates}
                candidates_bprob = {c:prob_bigrams(prev_word, c) for c in candidates}
#               Linear Interpolation
                candidates_prob = {k: lambda1 * candidates_uprob[k] + 
                               lambda2 * candidates_bprob[k] for k in candidates} 
                corrected_sentence.append(max(candidates_prob, key=candidates_prob.get))
        else:
            corrected_sentence.append(word)
        prev_word = word
        
    return corrected_sentence

print(correct2(['NIGEL', 'THRUSH', 'page', '48']))

['NIGEL', 'THRUSH', 'page', '48']


#### More ideas of improvement



1.   **Finding tense of the sentence:** Most of the errors after correcting each sentence by new model are because of tense. Hence by checking the grammer we can correct these errors. Tense can be identified by using the pos_tag and check the verbs for tense, but this is not an accurate method. We can either use nltk's Context Free Grammer (CFG) or language_check package which checks the grammer of the sentence and corrects it. 
2.   **Using Trigram model:** More symantics can be captured by using Trigram models, and using linear interpolation for all three with varying lambda values may give better accuracy
3. Collecting more training data will expand out dictionary and improves the accuracy of the model.





Repeat the evaluation (as in Task 5) of your new algorithm and show that it outperforms the algorithm from Task 3 and 4

In [0]:
def accuracy(test):
    # Write your code here

    n = 0.0
    n_correct = 0.0
    for sentence in test:
        n += len(sentence['corrected'])
        sentence_corrected = correct2(sentence['original'])
        for (word_a, word_c) in zip(sentence['corrected'], sentence_corrected):
            if word_a == word_c:
                n_correct += 1
            else:
               print(word_a + ' -- ' + word_c)

    print("{} words matched out of {} words in corrected sentences".format(n_correct, n))

    return n_correct/n

print("Accuracy of the current Spell Checker : {}".format(accuracy(test)))

goes -- go
bellringing -- bell_ringing
watch -- wash
second -- sexton
watch -- wash
watch -- each
cowboys -- cowboy
club -- come
watch -- each
watch -- wash
think -- thing
TV -- tv
square -- star
eyes -- yes
knocked -- nock
at -- a
killed -- kill
saw -- see
been -- bean
knocked -- nock
called -- cold
came -- cam
there -- the
Her -- Here
eyes -- yes
have_to -- have
else -- als
wheel -- week
wheel -- well
sallies -- sally
others -- other
ring -- rings
Cynthia's -- Cynthia
gas -- gass
masks -- mark
floor -- door
gas -- gass
masks -- mark
too -- to
what -- was
wash -- wish
perhaps -- tramp
ate -- it
soap -- some
gave -- cave
Bullimore -- bullock
gave -- cave
got -- go
beside -- side
her -- here
her -- here
soap -- some
what -- was
her -- here
gave -- cave
soap -- some
gave -- cave
that's -- that
killed -- kill
wait -- water
inject -- neck
make -- mack
sure -- shore
fork -- walk
knock -- nock
them -- the
worm -- worms
powder -- wonder
worm -- worms
get -- ge
their -- there
them -- theme
sur

In [0]:
'ring' in word_dictionary_lemma

True

By updating the spell checker with various improvements, an accuracy of 91.76% is achieved, which is clearly better model than the original spell checker (accuracy of 83.79%).